In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [2]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory automation' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [5]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Aniai, a robotic company and kitchen solutions provider based in New York City, recently secured $12 million in Pre-Series A funding. The funding round was led by InterVest, with contributions from SV Investment, Ignite Innovation from the UK, and Capstone Partners, an existing investor. The company plans to utilize the funds to establish its first manufacturing facility, Factory One, in South Korea. This move is aimed at ensuring stable production and efficient supply chain management for their flagship product, the Alpha Grill, as part of their expansion efforts in North America and globally.

Established in 2020 and helmed by CEO Gunpil Hwang, Aniai specializes in developing robotic solutions for cooking hamburgers. Their leading product, the Alpha Grill, utilizes robotics to cook hamburgers efficiently, featuring a double-sided grill capable of cooking up to eight patties simultaneously or 200 patties per hour. By automating tasks like pressing and flipping patties, the Alpha Grill

The Numbers Station, an intelligent data stack automation platform, recently announced significant funding of $17.5 million through a series A round led by Madrona. This funding round included a $12.5 million investment from Madrona, along with contributions from Norwest Venture Partners, Factory, and notable angel investors like Jeff Hammerbacher, the Co-founder of Cloudera. This funding follows a previous $5 million seed round, bringing the total capital raised by Numbers Station to $17.5 million. The platform utilizes foundation model technology developed at the Stanford AI lab to automate tedious data work, particularly in the realm of data transformation, enabling data workers to efficiently generate insights.

Source: Original text excerpt from an article on factory automation.

---

Numbers Station, a Menlo Park, CA-based company, recently secured $17.5 million in funding for its intelligent data stack automation platform. The funding came from a $12.5 million Series A round led

Software Defined Automation recently secured $10 million in seed funding, with Insight Partners leading the investment round. Other participants included Baukunst VC, Fly Ventures, and First Momentum. Jon Rosenbaum from Insight Partners and Axel Bichara from Baukunst will now join the company's Board of Directors as part of the deal.

The company specializes in revolutionizing factory automation through its Industrial-Control-as-a-Service (ICaaS) offering. This approach involves cloud-based management of existing Programmable Logic Controllers (PLCs) for technical operations (TechOps), Git-enabled PLC code versioning and collaboration for development operations (DevOps), and virtualization of PLCs on edge servers to create Virtual PLCs. By combining these technologies, Software Defined Automation aims to eliminate proprietary silos within control technology stacks and facilitate the adoption of modern microservices architecture through API integration.

The company operates from two ma

In [6]:
Problem = """Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry."""

print(Problem)

Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry.


# Answering questions on the results with OpenAI GPT

In [10]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f""" You are Senior Vice President Technology and Innovation & CTO. You were just handed the following problem:

       
        {Problem}
        
     When you evaluate a new business idea, you strictly adhere to the following guiding principles:
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets, avoiding investments over 10-15 million per portfolio element.
        - Target a minimum 10 million EUR annual revenue.
        - Specialize in LLMs in Engineering and Operations.
        - Avoid new ecosystem creation.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Aim for non-cannibalizing, fringe markets.

        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

budgets = [1000, 10000, 100000,1000000,10000000]  # Budget constraints for each solution proposal
summary_solutions = []

user_prompt = f"""
        As an input, you are give the {Problem}.
        For each budget USD {budgets}, come up with one different solution, each tailored to this specific budget. 
        The solutions should be innovative and be achievable within the budget.
        For each budget, you should leverage insights or approaches mentioned in the following document summary:

        Document summary:
        ```
        {summary_list}
        ```

        Please make sure you use the budgets that were set correctly, and that the idea is really achievable within that budget.
      
     
        Please summarize in one paragraph for each solution, with the following content:
    
        “Name”: Solution name (5 words max)
        “Solved Problem”: 1 sentence summary of the problem that is being solved and how much budget is needed
        “Solution”: Provide a short summary of the solution and how it works (3 sentences max)
        “Impact”: How does the solution solve the specific problem (3 sentences max)
        “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution  (5 sentences max)
        “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    

    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions.append(completion.choices[0].message.content)


**1,000 USD Solution**

Name: Basic Integration Guide

Solved Problem: Addressing the communication gap between factory automation systems on a minimal budget of 1,000 USD.

Solution: Develop a comprehensive, beginner-friendly guide and checklist for factory automation engineers and operators. This guide will include best practices, standard protocols for equipment integration, and troubleshooting steps to enhance system communication with minimal investment. It will be distributed as a digital document available for download.

Impact: This solution empowers engineers and operators to initiate basic integration efforts, reducing downtime and improving workflow efficiency through better understanding and application of existing protocols and practices.

Technology: The guide will be compiled using insights from existing successful integration strategies, such as those employed by Software Defined Automation in revolutionizing factory automation through Industrial-Control-as-a-Service (I

In [ ]:
# Initial rating of the solutions

initial_rating = []

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """

user_prompt_2 = f""""

Please take all solutions from the following text: {summary_solutions}
First, repeat the following points for the first solution:
    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt_2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message



In [ ]:
user_prompt_3 = f""""

Please take all solutions and the corresponding ratings from: {initial_rating}

Take the three solutions with the highest overall rating out of 15. 

Outline the following points for these solution:
    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt_2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability